## Online News Popularity Data Set


> Mallios Charalampos, Student - p2821912 <br />
> Department of Management Science and Technology <br />
> Msc Business Analytics <br />
> Athens University of Economics and Business <br />
> p2821912@aueb.gr

* First step was to initalize Spark and set memory boundaries.
* Then we created the schema, according to the data description and type and read the dataset.


In [1]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.types import *
from pyspark import SparkConf
#initialize Spakr session
import findspark
findspark.init()
from pyspark.sql import SparkSession
# create app and set memory range in the executors and driver in order not to overspend it.
spark =  SparkSession.builder.appName("online-popularity-news").config("spark.executor.memory", "6g").config("spark.driver.memory", "5g").getOrCreate()


In [2]:
from pyspark.sql.types import StructType

#define the schema , for each feature the correct datatype
schema = StructType([
StructField("url", StringType(), True),
StructField("timedelta", DoubleType(), True),
StructField("n_tokens_title", DoubleType(), True),
StructField("n_tokens_content", DoubleType(), True),
StructField("n_unique_tokens", DoubleType(), True),
StructField("n_non_stop_words", DoubleType(), True),
StructField("n_non_stop_unique_tokens", DoubleType(), True),
StructField("num_hrefs", DoubleType(), True),
StructField("num_self_hrefs", DoubleType(), True),
StructField("num_imgs", DoubleType(), True),
StructField("num_videos", DoubleType(), True),
StructField("average_token_length", DoubleType(), True),
StructField("num_keywords", DoubleType(), True),
StructField("data_channel_is_lifestyle", DoubleType(), True),
StructField("data_channel_is_entertainment", DoubleType(), True),
StructField("data_channel_is_bus", DoubleType(), True),
StructField("data_channel_is_socmed", DoubleType(), True),
StructField("data_channel_is_tech", DoubleType(), True),
StructField("data_channel_is_world", DoubleType(), True),
StructField("kw_min_min", DoubleType(), True),
StructField("kw_max_min", DoubleType(), True),
StructField("kw_avg_min", DoubleType(), True),
StructField("kw_min_max", DoubleType(), True),
StructField("kw_max_max", DoubleType(), True),
StructField("kw_avg_max", DoubleType(), True),
StructField("kw_min_avg", DoubleType(), True),
StructField("kw_max_avg", DoubleType(), True),
StructField("kw_avg_avg", DoubleType(), True),
StructField("self_reference_min_shares", DoubleType(), True),
StructField("self_reference_max_shares", DoubleType(), True),
StructField("self_reference_avg_sharess", DoubleType(), True),
StructField("weekday_is_monday", DoubleType(), True),
StructField("weekday_is_tuesday", DoubleType(), True),
StructField("weekday_is_wednesday", DoubleType(), True),
StructField("weekday_is_thursday", DoubleType(), True),
StructField("weekday_is_friday", DoubleType(), True),
StructField("weekday_is_saturday", DoubleType(), True),
StructField("weekday_is_sunday", DoubleType(), True),
StructField("is_weekend", DoubleType(), True),
StructField("LDA_00", DoubleType(), True),
StructField("LDA_01", DoubleType(), True),
StructField("LDA_02", DoubleType(), True),
StructField("LDA_03", DoubleType(), True),
StructField("LDA_04", DoubleType(), True),
StructField("global_sentiment_polarity", DoubleType(), True),
StructField("global_subjectivity", DoubleType(), True),
StructField("global_rate_positive_words", DoubleType(), True),
StructField("global_rate_negative_words", DoubleType(), True),
StructField("rate_positive_words", DoubleType(), True),
StructField("rate_negative_words", DoubleType(), True),
StructField("avg_positive_polarity", DoubleType(), True),
StructField("min_positive_polarity", DoubleType(), True),
StructField("max_positive_polarity", DoubleType(), True),
StructField("avg_negative_polarity", DoubleType(), True),
StructField("min_negative_polarity", DoubleType(), True),
StructField("max_negative_polarity", DoubleType(), True),
StructField("title_subjectivity", DoubleType(), True),
StructField("title_sentiment_polarity", DoubleType(), True),
StructField("abs_title_subjectivity", DoubleType(), True),
StructField("abs_title_sentiment_polarity", DoubleType(), True),
StructField("shares", DoubleType(), True)])

#read the data and stored as they were on schema defined
data = spark\
  .read\
  .option("header", "true")\
  .schema(schema)\
  .csv("OnlineNewsPopularity.csv")
#see the schema
data.printSchema()
# see the first 5 rows
data.limit(5).toPandas()

root
 |-- url: string (nullable = true)
 |-- timedelta: double (nullable = true)
 |-- n_tokens_title: double (nullable = true)
 |-- n_tokens_content: double (nullable = true)
 |-- n_unique_tokens: double (nullable = true)
 |-- n_non_stop_words: double (nullable = true)
 |-- n_non_stop_unique_tokens: double (nullable = true)
 |-- num_hrefs: double (nullable = true)
 |-- num_self_hrefs: double (nullable = true)
 |-- num_imgs: double (nullable = true)
 |-- num_videos: double (nullable = true)
 |-- average_token_length: double (nullable = true)
 |-- num_keywords: double (nullable = true)
 |-- data_channel_is_lifestyle: double (nullable = true)
 |-- data_channel_is_entertainment: double (nullable = true)
 |-- data_channel_is_bus: double (nullable = true)
 |-- data_channel_is_socmed: double (nullable = true)
 |-- data_channel_is_tech: double (nullable = true)
 |-- data_channel_is_world: double (nullable = true)
 |-- kw_min_min: double (nullable = true)
 |-- kw_max_min: double (nullable = tru

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
0,http://mashable.com/2013/01/07/amazon-instant-...,731.0,12.0,219.0,0.663594,1.0,0.815385,4.0,2.0,1.0,...,0.100000,0.7,-0.350000,-0.600,-0.200000,0.500000,-0.187500,0.000000,0.187500,593.0
1,http://mashable.com/2013/01/07/ap-samsung-spon...,731.0,9.0,255.0,0.604743,1.0,0.791946,3.0,1.0,1.0,...,0.033333,0.7,-0.118750,-0.125,-0.100000,0.000000,0.000000,0.500000,0.000000,711.0
2,http://mashable.com/2013/01/07/apple-40-billio...,731.0,9.0,211.0,0.575130,1.0,0.663866,3.0,1.0,1.0,...,0.100000,1.0,-0.466667,-0.800,-0.133333,0.000000,0.000000,0.500000,0.000000,1500.0
3,http://mashable.com/2013/01/07/astronaut-notre...,731.0,9.0,531.0,0.503788,1.0,0.665635,9.0,0.0,1.0,...,0.136364,0.8,-0.369697,-0.600,-0.166667,0.000000,0.000000,0.500000,0.000000,1200.0
4,http://mashable.com/2013/01/07/att-u-verse-apps/,731.0,13.0,1072.0,0.415646,1.0,0.540890,19.0,19.0,20.0,...,0.033333,1.0,-0.220192,-0.500,-0.050000,0.454545,0.136364,0.045455,0.136364,505.0


* Fields 'url' and 'timedelta' that were not predictive were dropped.
* After that the dataset splitted into training and test with 0.75/0.35 ratio.


In [4]:

# drop the non - predictive features
data = data.drop('url')
data = data.drop('timedelta')

#split to train and test
(training_data, test_data) = data.randomSplit([0.75, 0.25])



* We created the vector with all the x features with Vector assembler.
* Created the pipeline, define Random Forest Regresson with different tuning parameters.
* And after cross validation we choose the best model and test it on test dataset.
* Finally RMSE along with R-squared were printed to evaluate the predictions, together with the statistics of Y variable 'shares'.

In [6]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder,TrainValidationSplit
from pyspark.ml.regression import RandomForestRegressor


# create the vector features containing the features data all together(x) except Y(shares)
assembler = VectorAssembler(
    inputCols=[ x for x in training_data.columns if x != "shares" ],
    outputCol="features")



#define the random forest regressor
rf = RandomForestRegressor().\
    setLabelCol("shares").\
    setFeaturesCol("features").\
    setPredictionCol("prediction")

#set different tuning hyperparameters levels for optimization
paramGrid = ParamGridBuilder()\
  .addGrid(rf.featureSubsetStrategy, ['log2','sqrt','onethird']) \
  .addGrid(rf.numTrees, [100,500,700]) \
  .build()

#define the pipeline contains the steps 
pipeline = Pipeline().setStages([assembler,rf])

#define the evaluator of regresson - based on RMSE
evaluator = RegressionEvaluator(
    labelCol="shares", predictionCol="prediction", metricName="rmse")
evaluator2 = RegressionEvaluator(
    labelCol="shares", predictionCol="prediction", metricName="r2")



#make Cross validation to tune our regressor best parameters
tvs = TrainValidationSplit(estimator=pipeline,
                           estimatorParamMaps=paramGrid,
                           evaluator=evaluator,
                           # 80% of the data will be used for training, 20% for validation.
                           trainRatio=0.8)

#fit the model
model = tvs.fit(training_data)

#make the prediction on test data
predictions = model.bestModel.transform(test_data)

# extract for the best model selected the tuning hyperparameters
params = model.bestModel.stages[-1].extractParamMap()

for param, value in params.items():
        print(f'{param.name}: {value}')
rmse = evaluator.evaluate(predictions)
r2 = evaluator2.evaluate(predictions)

#extract RMSE and R^2 from our model prediction
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)
print("R2 on test data = %g" % r2)




cacheNodeIds: False
checkpointInterval: 10
featureSubsetStrategy: sqrt
featuresCol: features
impurity: variance
labelCol: shares
maxBins: 32
maxDepth: 5
maxMemoryInMB: 256
minInfoGain: 0.0
minInstancesPerNode: 1
numTrees: 100
predictionCol: prediction
seed: -3375131582288097643
subsamplingRate: 1.0
Root Mean Squared Error (RMSE) on test data = 8016.32
R2 on test data = 0.0176969


In [8]:
#select shares and get descriptive statistics of target variable
shares = data.select('shares')
shares.describe().show()

+-------+------------------+
|summary|            shares|
+-------+------------------+
|  count|             39644|
|   mean|3395.3801836343455|
| stddev| 11626.95074865171|
|    min|               1.0|
|    max|          843300.0|
+-------+------------------+



* **RMSE** value (8016.32) is **not** good in terms of predictive performance. The value is lower than Standard Deviation of the shares(Y variable), although is much higher than the mean of target variable 3395. Morevoer, regarding R-squared its performance is also poor (0.017), sice R-squared defines how well the regression predictions approximate the real data points.
    
* A metric to better evaluate the **RMSE** is **Scatter Index** which is defined as : **RMSE / mean(Y)**. This value must be below < 1 but in our case is much higher 8016/3395(~2.36). 

* In articles with great number of shares (e.x. > 100.000) our model would be ok in terms of prediction error. Now, with the average shares being around 3395 shares, the 8016 shares error is not acceptable.

* We can observe the reason behind that : our dataset is **imbalanced** and the distribution has skewness. The max value of 843300 is far away from the mean (3395) and this has huge impact on our prediction effort , since some **outliers** like the max value have affect our results.

* To prove that we get the 99% quantile to explore what is happening :

In [9]:
outliers = data.approxQuantile("shares", [0.99], 0)
outliers

[31700.0]

* Indeed, we see that 99% of observations are below 31700. 

### Suggest a different approach based on Correlation

* We should also check for the **correlation** between X variables and Y ('shares').
* In a second try to find a better predictive model we should make variable selection based on correlation and keep 
only those which make impact in a new dataset.
* So below we repeat the process again , now having only the high **correlated data from dataset** and making some scaling on data to see if it will help on normalization.

In [10]:
from pyspark.ml.stat import Correlation


data = data.drop('url')
data = data.drop('timedelta')
#select all columns to keep the names
correlat = data.select('*')

#enumerate on column names and print the correlation with Y
names = correlat.schema.names
for name in names:
  print(name)
  print(correlat.stat.corr(name,"shares"))

#The newdataset contains only high correlated variables    
new = correlat.select('num_hrefs','num_imgs','data_channel_is_world','kw_avg_max','kw_min_avg','kw_max_avg','kw_avg_avg','self_reference_min_shares','self_reference_avg_sharess','LDA_02','LDA_03','shares')

#split to train and test
(training_data, test_data) = new.randomSplit([0.75, 0.25])



n_tokens_title
0.008783118841819185
n_tokens_content
0.002458984345090757
n_unique_tokens
0.0008063456613076668
n_non_stop_words
0.00044294161730838635
n_non_stop_unique_tokens
0.00011417193781090629
num_hrefs
0.04540400840592801
num_self_hrefs
-0.0019004033728368833
num_imgs
0.03938759784762878
num_videos
0.023936069530130968
average_token_length
-0.02200729806193876
num_keywords
0.021818227151875386
data_channel_is_lifestyle
0.005831267306546816
data_channel_is_entertainment
-0.017006198429558247
data_channel_is_bus
-0.012376166187935982
data_channel_is_socmed
0.005021216306495134
data_channel_is_tech
-0.013252874428024544
data_channel_is_world
-0.049497308931198525
kw_min_min
-0.0010509877309311786
kw_max_min
0.030113936670928126
kw_avg_min
0.03040554741568556
kw_min_max
0.0039014319770320736
kw_max_max
0.007862569278242268
kw_avg_max
0.04468584480857034
kw_min_avg
0.039550694479642406
kw_max_avg
0.06430586384745443
kw_avg_avg
0.11041286270954459
self_reference_min_shares
0.05595757

In [10]:
# check if they are stacked correctly
training_data.limit(5).toPandas()

,num_hrefs,num_imgs,data_channel_is_world,kw_avg_max,kw_min_avg,kw_max_avg,kw_avg_avg,self_reference_min_shares,self_reference_avg_sharess,LDA_02,LDA_03,shares
0,0.0,0.0,0.0,16080.000000,0.0,2939.239130,1596.149445,0.0,0.0,0.020019,0.020554,1700.0
1,0.0,0.0,0.0,17255.555556,0.0,4380.000000,2328.023284,0.0,0.0,0.260606,0.023187,4500.0
2,0.0,0.0,0.0,17628.000000,440.0,4423.333333,2305.412167,0.0,0.0,0.020075,0.918569,1000.0
3,0.0,0.0,0.0,18180.000000,0.0,3169.658537,2035.970551,0.0,0.0,0.020007,0.020177,1500.0
4,0.0,0.0,0.0,18442.857143,0.0,2810.766129,1415.501897,0.0,0.0,0.028589,0.660815,589.0


In [14]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder,TrainValidationSplit
from pyspark.ml.tuning import TrainValidationSplit
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import StandardScaler


# making some scaling on data to see if it helps
scaler = StandardScaler(inputCol="features_ass", outputCol="features",
                        withStd=True, withMean=True)
#vector assembler again
assembler = VectorAssembler(
    inputCols=[ x for x in training_data.columns if x != "shares" ],
    outputCol="features_ass")

#define random forest regressor
rf = RandomForestRegressor().\
    setLabelCol("shares").\
    setFeaturesCol("features").\
    setPredictionCol("prediction")

# parameters tuning and their ranges
paramGrid = ParamGridBuilder()\
  .addGrid(rf.featureSubsetStrategy, ['log2','sqrt','onethird']) \
  .addGrid(rf.numTrees, [100,500,700]) \
.build()

#set the pipeline with the 3 stages -> assemberl, scaler, rf regressor
pipeline = Pipeline().setStages([assembler,scaler,rf])

#evaluate the results in terms of RMSE
evaluator = RegressionEvaluator(
    labelCol="shares", predictionCol="prediction", metricName="rmse")

#make Cross validation to tune our regressor best parameters
tvs = TrainValidationSplit(estimator=pipeline,
                           estimatorParamMaps=paramGrid,
                           evaluator=evaluator,
                           # 80% of the data will be used for training, 20% for validation.
                           trainRatio=0.8)
#fit on training data
model = tvs.fit(training_data)

#apply the best model on test data
predictions = model.bestModel.transform(test_data)


# extract for the best model selected the tuning hyperparameters
params = model.bestModel.stages[-1].extractParamMap()

for param, value in params.items():
        print(f'{param.name}: {value}')

        
#extract RMSE and R^2 from our model prediction
rmse = evaluator.evaluate(predictions)
r2 = evaluator2.evaluate(predictions)


print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)
print("R2 on test data = %g" % r2)




cacheNodeIds: False
checkpointInterval: 10
featureSubsetStrategy: log2
featuresCol: features
impurity: variance
labelCol: shares
maxBins: 32
maxDepth: 5
maxMemoryInMB: 256
minInfoGain: 0.0
minInstancesPerNode: 1
numTrees: 500
predictionCol: prediction
seed: -3375131582288097643
subsamplingRate: 1.0
Root Mean Squared Error (RMSE) on test data = 10582.4
R2 on test data = 0.021219


* We see that the results were slightly different. **RMSE** was decreased to 10582.4 and **R-squared** was slightly increased to 0.021 , however still in not such an effective way.

* So, beside the fact we got a different approach to variable selection the results are not better in terms of predictibe ability.

**In a nutshell**, we can see that the Random Forest Regressor has not good predictive ability on our dataset. 
The main reason behind that, is the dataset. The data are inbalanced and the distribution of shares is not normal.
Moreover,after applying correlation between every single feauture and Y we saw that there is only some feautures with correlation,but not in such an attentive way , in particarly with absolute correlation above or below 30. So, based on this datset and data distribution we can't expect a better result.